<a href="https://colab.research.google.com/github/kacperwojtyniak/ml-playground/blob/main/Notebooks/Damaged_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

Set bing key

In [ ]:
key = os.environ.set('AZURE_SEARCH_KEY', '')

Specify search terms

# Search for images

In [ ]:
search_terms = {'damaged':'car dent', 'undamaged':'Mid-Size Car'}
path = Path('cars')

Download images

In [ ]:
if not path.exists():
  path.mkdir()
  
for k in search_terms:
  search_term = search_terms[k]
  dest = (path/k)
  dest.mkdir(exist_ok=True)
  results = search_images_bing(key, search_term) #max 150 images
  download_images(dest, urls=results.attrgot('contentUrl'))

Cleanup broken images

In [ ]:
fns = get_image_files(path)
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

Create datablock

# Train

In [ ]:
# path = Path('gdrive/MyDrive/Colab Notebooks/datasets/cars')

In [ ]:
cars = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=35),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.4))
dls = cars.dataloaders(path)
dls.valid.show_batch(max_n=12, nrows=3)

In [ ]:
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [ ]:
learn.fine_tune(5)

In [ ]:
learn.export()

In [ ]:
path = Path()
learn_inf = load_learner(path/'export.pkl')

In [ ]:
test_url = 'https://ireland.apollo.olxcdn.com/v1/files/eyJmbiI6ImgxbTl3YzNsZmVwZTItT1RPTU9UT1BMIiwidyI6W3siZm4iOiJ3ZzRnbnFwNnkxZi1PVE9NT1RPUEwiLCJzIjoiMTYiLCJwIjoiMTAsLTEwIiwiYSI6IjAifV19.z-yaN24dt7FTMzsA02dm9O_-oMYEMyg9jnXNI-O6jjM/image;s=1080x720'
tst_dest = './tst.jpg'
download_url(test_url, tst_dest)

In [ ]:
learn_inf.predict('./tst.jpg')